<a href="https://colab.research.google.com/github/turneyj37/turneyj37/blob/main/Supernova_Fit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The first objective of this project is to convert Arnett's equations from 1982 into Python.  The following functions are set to do just that.

In [ ]:
### IMPORTING PACKAGES ###
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as sp
import astropy.modeling.models as ap
import pandas as pd
from scipy.integrate import quad


### First, we need to set some constants for use in these expressions:
c = 299792458 ### the speed of light, in meters per second
kappa = 0.6 ### an average value for the opacity, stated in Arnett, 1980
tau_ni = 6.077 * 24 * 3600  ### the half life of nickel 56 in seconds
e_ni = 3.42 * 10**(-6)  ### the energy released from nickel 56 decay

def tau_0(k, M_e, R_0):
  return (kappa)*(M_e)/(13.7 * c * R_0) ### The diffusion timescale;
### R_0 is initial radius, kappa is opacity, M_e is the total mass ejected

def v_sc(M_n, M_e):
  return (2.53 * (M_n)/(M_e))**(1/2) ### The scale velocity, dependent on the 
### mass of nickel ejected and the total mass ejected

def tau_m(R_0, k, M_e, M_n): 
  return ((2 * R_0 * tau_0(k, M_e, R_0))/(v_sc(M_n,M_e)))**(1/2) ### The mean timescale;
### A geometric mean of the expansion and diffusion timescales

### Now we can begin to set the parameters for the lambda function:

def v_1(R_0, k, M_e, M_n):
  return x/tau_m(R_0, k, M_e, M_n) 
### this is the equivalent of the x variable in Arnett, 1982, and
### the x inside the function is time, which will be along the x axis come time 
### for the analysis.

def v_2(R_0, k, M_e, M_n): 
  return tau_m/(2 * tau_ni)  ### this is the equivalent of the y variable in Arnett
### 1982;

def lambda_int(i):
  return exp(-2*z*i + z**2) * 2*z
### This expression resides inside the integral for the lambda family of curves
### z is merely a dummy variable

### Evaluating the lambda family of curves:
def lambda_fam(x, y):
  return exp(-(x ** 2)) * quad(lambda_int(y),0,x)
### The limit of the integration is in fact the first variable.

### Now we define the Deposition function and the coefficients:
def D_fn_gamma(x, y, k, M_n, M_e):
  return G_gamma(x, y, k, M_n, M_e) * (1 + 2*G_gamma(x, y, k, M_n, M_e)*\
          (1-G_gamma(x, y, k, M_n, M_e))*(1 - 0.75*G_gamma(x, y, k, M_n, M_e)))
def D_fn_pos(x, y, k, M_n, M_e):
  return G_pos(x, y, k, M_n, M_e) * (1 + 2*G_pos(x, y, k, M_n, M_e)*\
          (1-G_pos(x, y, k, M_n, M_e))*(1 - 0.75*G_pos(x, y, k, M_n, M_e)))
### This is the form presented in Arnett, 1980

def tau_gamma(x, y, k, M_n, M_e): ### This allows me to change x and y to investigate 
  return (5.53 * y**2)/(k * v_sc(M_n, M_e) * (0.1 +2*x*y)**2)
def tau_pos (x, y, k, M_n, M_e):
  return 355*tau_gamma(x, y, k, M_n, M_e)
### These are the optical depths for photons (gamma) and positrons (pos)
 
def G_gamma(x, y, k, M_n, M_e):
  return tau_gamma(x, y, k, M_n, M_e)/(tau_gamma(x, y, k, M_n, M_e) + 1.6) ### I will later vary x and 
### y to investigate how the luminosity profile changes with these variables.
def G_pos(x, y, k, M_n, M_e):
  return tau_pos(x, y, k, M_n, M_e)/(tau_pos(x, y, k, M_n, M_e) + 1.6)
### These are the individual factors of the deposition function

### We need to include the lambda family of functions for cobalt as well:
def lambda_co(x,y): 
  return 5.36 * 10**(-3) * exp(-0.1548*x*y)

### Now we can put it all together:

def lum_tot(x, y, k, M_n, M_e, R_0):
  return e_n*M_n*lambda_fam(x,y)*D_fn_gamma(x, y, k, M_n, M_e) + \
  lambda_co(x,y)*(0.81*D_fn_gamma(x, y, k, M_n, M_e) + \
                  0.19*D_fn_pos(x, y, k, M_n, M_e))

